# TOC
- [Demo](#demo)
- [Variance](#variance)
- [Type projection](#type&projection)
- [Generic functions](#generic&functions)
- [Type erasure](#type&erasure)
- [](#)
- [](#)

## Demo

In [3]:
class Box<T>(t: T) {
    var value = t
    
    override fun toString(): String {
        return "Box = {value = $value}"
    }
}

In [4]:
val b: Box<Int> = Box<Int>(1)
val box = Box(1)
val box2 = Box("1")

println(b.toString())
println(box.value.javaClass)
println(box2.value.javaClass)

Box = {value = 1}
class java.lang.Integer
class java.lang.String


## Variance

In [5]:
interface Source<out T> {
    fun nextT(): T
}

fun <T> demo(source: Source<T>) {
    val objects: List<T> = List(3) { source.nextT() } // Creating a List from the source
    for (item in objects) {
        println(item)
    }
}

class StringSource(private val data: List<String>) : Source<String> {
    private var index = 0

    override fun nextT(): String {
        if (index < data.size) {
            return data[index++]
        }
        throw NoSuchElementException("No more elements")
    }
}

class NumberSource(private val data: List<Int>) : Source<Int> {
    private var index = 0

    override fun nextT(): Int {
        if (index < data.size) {
            return data[index++]
        }
        throw NoSuchElementException("No more elements")
    }
}

In [6]:
val sourceOfString: Source<String> = StringSource(listOf("apple", "banana", "cherry"))
demo(sourceOfString)

val numberSource: Source<Int> = NumberSource(listOf(1, 2, 3))
demo(numberSource)

apple
banana
cherry
1
2
3


In [7]:
interface Comparable<in T> {
    operator fun compareTo(other: T): Int
}

class Person(val name: String, val age: Int) : Comparable<Person> {
    override fun compareTo(other: Person): Int {
        return when {
            age < other.age -> -1
            age > other.age -> 1
            else -> 0
        }
    }
}

In [8]:
val person1 = Person("Alice", 25)
val person2 = Person("Bob", 30)
val person3 = Person("Charlie", 30)

println(person1.compareTo(person2))
println(person2.compareTo(person3))

-1
0


## Type projection

In [9]:
fun copy(from: Array<out Any>, to: Array<Any>) {
    assert(from.size == to.size)
    for (i in from.indices) {
        to[i] = from[i]
    }
}

In [10]:
val intArray: Array<Int> = arrayOf(1, 2, 3)
val anyArray = Array<Any>(3) { "" }

copy(from = intArray, to = anyArray)

intArray.forEach { print(it) }
anyArray.forEach { print(it) }

123123

In [11]:
fun fill(dest: Array<in String>, value: String) {
    dest[0] = value
}

In [12]:
// Example 1: Using Array<String>
val arrayOfString: Array<String> = arrayOf("a", "b", "c")
fill(arrayOfString, "test")

println("Example 1: Array<String> after fill: ${arrayOfString.joinToString()}")

Example 1: Array<String> after fill: test, b, c


In [13]:
// Example 2: Using Array<CharSequence>
val arrayOfCharSequence: Array<CharSequence> = arrayOf("x", "y", "z")
fill(arrayOfCharSequence, "test")

println("Example 2: Array<CharSequence> after fill: ${arrayOfCharSequence.joinToString()}")

Example 2: Array<CharSequence> after fill: test, y, z


In [14]:
// Example 3: Using Array<Any>
val arrayOfAny: Array<Any> = arrayOf("1", "2", "3")
fill(arrayOfAny, "test")

println("Example 3: Array<Any> after fill: ${arrayOfAny.joinToString()}")

Example 3: Array<Any> after fill: test, 2, 3


## Generic functions

In [15]:
fun <T> singletonList(item: T): List<T> {
    return listOf(item)
}

fun <T> T.basicToString(): String {
    return this.toString()
}

In [16]:
val intList = singletonList(10)
println("Singleton List of Integers: $intList")

val stringList = singletonList("Hello")
println("Singleton List of Strings: $stringList")

val num = 42
println("Basic toString of $num: ${num.basicToString()}")

val str = "Kotlin"
println("Basic toString of $str: ${str.basicToString()}")

Singleton List of Integers: [10]
Singleton List of Strings: [Hello]
Basic toString of 42: 42
Basic toString of Kotlin: Kotlin


## Type erasure

In [17]:
inline fun <reified A, reified B> Pair<*, *>.asPairOf(): Pair<A, B>? {
    if (first !is A || second !is B) return null
    return first as A to second as B
}

val somePair: Pair<Any?, Any?> = "items" to listOf(1, 2, 3)

val stringToSomething = somePair.asPairOf<String, Any>()
val stringToInt = somePair.asPairOf<String, Int>()
val stringToList = somePair.asPairOf<String, List<*>>()
val stringToStringList = somePair.asPairOf<String, List<String>>() // Compiles but breaks type safety!

In [18]:
println("stringToSomething = " + stringToSomething)
println("stringToInt = " + stringToInt)
println("stringToList = " + stringToList)
println("stringToStringList = " + stringToStringList)

stringToSomething = (items, [1, 2, 3])
stringToInt = null
stringToList = (items, [1, 2, 3])
stringToStringList = (items, [1, 2, 3])


## Underscore operator for type arguments

In [19]:
abstract class SomeClass<T> {
    abstract fun execute() : T
}

class SomeImplementation : SomeClass<String>() {
    override fun execute(): String = "Test"
}

class OtherImplementation : SomeClass<Int>() {
    override fun execute(): Int = 42
}

object Runner {
    inline fun <reified S: SomeClass<T>, T> run() : T {
        return S::class.java.getDeclaredConstructor().newInstance().execute()
    }
}

In [20]:
// T is inferred as String because SomeImplementation derives from SomeClass<String>
val s = Runner.run<SomeImplementation, _>()
assert(s == "Test")

// T is inferred as Int because OtherImplementation derives from SomeClass<Int>
val n = Runner.run<OtherImplementation, _>()
assert(n == 42)